In [147]:
import networkx as nx
from networkx.algorithms import bipartite
import numpy as np
import random
from scipy.stats import spearmanr

# Loading data pathes and I/O functions from script
from scripts.io import load_movie_titles, load_raw_bipartite, save_projection, load_projection, save_edgelist, projection_path

# Loading reccomendation function
from scripts.recommend import evaluation_recommendation, sort_average_weight

ImportError: cannot import name 'evaluation_recommendation' from 'scripts.recommend' (C:\Users\Christian\OneDrive\ITU\Network Analysis\Project\Networks\scripts\recommend.py)

## Loading dicts and graphs

In [157]:
title_dict, node_dict = load_movie_titles("movie-titles.txt")

G = load_raw_bipartite("full_bipartite.p")

simple_weights_movies = load_projection("simple_weights_movies.p")
rating_allocation_movies = load_projection("rating_allocation_movies.p")

Movie titles loaded.
Graph loaded.
Projection loaded.
Projection loaded.


# Evaluation
    Cross-validation of movie recommendations from graph with empirical ratings by users.

    For each user:
        1. Sample k liked (highest rated) movies, M, from list of liked movies, L ((movie_node, weight) list).
        2. Get movie recommendations, R ((movie_node, weight) list), for all movie nodes, based on M.
        3. Sort R and L by movie_node, then discard movie_node from both.
        5. Compute and store spearman rank correlation between R and L.
        
    Output average rank correlation for all users.

In [156]:
def evaluate(movie_graph, k=3, G=G, seed=None):
    """
    Cross-validation of movie recommendations from graph with empirical ratings of users.
    
    params:
        movie_graph (nx.Graph or nx.DiGraph) graph from which to recommend movies by highest weight neighbors
        k (int) number of liked movies to sample from each user for recommendation
        G (nx.Graph) bipartite graph of users and movies
    
    returns:
        average_correlation 
    """
    
    # Optional reproducibility of evaluation using random seed
    if seed:
        random.seed(seed)

    user_nodes, movie_nodes = nx.algorithms.bipartite.basic.sets(G)

    # list of correlation of each user's ratings and recommendations
    correlations = []
    
    # Proportion of correlations which have p>0.05
    not_significant_correlations = 0

    # For each user
    for user_node in user_nodes:

        ##### 1. Sample k liked (highest rated) movies
        
        # All rated movies and ratings of user
        movie_rating_tuples = [(movie_node,attr_dict['weight']) for movie_node, attr_dict in dict(G[user_node]).items()]  
        
        # Movies are shuffled, to randomize order of movies with the same rating
        random.shuffle(movie_rating_tuples)  
        
        # Movies are sorted by rating
        movie_rating_tuples = sorted(movie_rating_tuples, reverse=True, key=lambda x:x[1])  

        # k highest rated movies are sampled
        liked_movie_nodes = [movie_node for movie_node, weight in movie_rating_tuples[:k]]  

        ##### 2. Get movie recommendations
        
        recommended_movie_nodes = evaluation_recommendation(movie_graph, liked_movie_nodes)

        # Removing sampled nodes from liked movies
        liked_movie_nodes = [node for node in liked_movie_nodes if node not in liked_movie_nodes]
        assert len(liked_movie_nodes) == len(recommended_movie_nodes)
    
        ##### 3. Sort R and L by node
        liked_movie_nodes = sorted(liked_movie_nodes, key=lambda x:x[0])
        recommended_movie_nodes = sorted(recommended_movie_nodes, key=lambda x:x[0])
        
        ##### 4. Compute and store spearman rank correlation (and p value) between R and L.
        r, _ = spearmanr(liked_movie_nodes, recommended_movie_nodes)
        if p > 0.05:
            r = 0
        correlations.append(r)

    ##### Print average correlation between recommendations and preferences.
    print(f"Correlation: {np.mean(correlations):.3f}, sd: {np.std(correlations):.3f}, not-significant: {np.sum(correlations==0):.3f%}")

In [158]:
evaluate(simple_weights_movies, k=3, seed=0)

NameError: name 'evaluation_recommendation' is not defined